In [1]:
# !unzip -uq "dataset_4m.zip" -d "dataset_4m/"

In [2]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [4]:
path_plus='dataset_4m/dataset_4m/E01' # 무표정
path_zero='dataset_4m/dataset_4m/E02' # 웃음
path_minus='dataset_4m/dataset_4m/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:11520
plus 경로에 저장된 파일의 개수:11520
minus 경로에 저장된 파일의 개수:11520


In [16]:
xsize=100 # size 100*100 => 200*200으로 변경
ysize=100

In [17]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, grayscale=False, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [18]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [19]:
x=np.array(x)
y=np.array(y)

In [20]:
print(x.shape)

(34560, 100, 100, 3)


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

input_shape = (xsize,ysize, 3)

# batch_size = 32
# num_classes = 2
# epochs = 15

In [22]:
print(x_test.shape)
print(y_test.shape)

(13824, 100, 100, 3)
(13824,)


In [23]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications.mobilenet_v2 import MobileNetV2  
from keras.applications.mobilenet import MobileNet

input_shape = (100, 100, 3)
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model.trainable=False

x = Flatten()(base_model.output)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 101, 101, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  del sys.path[0]


In [25]:
history = model.fit(x_train, y_train, validation_split=0.5, 
          epochs=30, batch_size=256, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)

Train on 10368 samples, validate on 10368 samples
Epoch 1/30
10368/10368 [==============================] - 29s 3ms/step - loss: 0.8152 - accuracy: 0.7308 - val_loss: 6.0337 - val_accuracy: 0.5642
Epoch 2/30
10368/10368 [==============================] - 21s 2ms/step - loss: 0.1685 - accuracy: 0.9339 - val_loss: 5.2546 - val_accuracy: 0.6737
Epoch 3/30
10368/10368 [==============================] - 22s 2ms/step - loss: 0.0832 - accuracy: 0.9698 - val_loss: 3.9485 - val_accuracy: 0.7098
Epoch 4/30
10368/10368 [==============================] - 21s 2ms/step - loss: 0.0469 - accuracy: 0.9827 - val_loss: 3.5528 - val_accuracy: 0.7465
Epoch 5/30
10368/10368 [==============================] - 22s 2ms/step - loss: 0.0338 - accuracy: 0.9866 - val_loss: 7.0211 - val_accuracy: 0.6402
Epoch 6/30
10368/10368 [==============================] - 21s 2ms/step - loss: 0.0331 - accuracy: 0.9889 - val_loss: 10.4800 - val_accuracy: 0.5530
Epoch 7/30
10368/10368 [==============================] - 22s 2ms/s

In [26]:
model.save('save/MobileNetV2(40000).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [17]:
# !unzip -uq "roi_1.zip" -d "dataset_pred/"

In [27]:
test_path = "dataset_pred/E01" # 기쁨 
test_path2 = "dataset_pred/E02" # 무표정 
test_path3 = "dataset_pred/E03" # 찡그림

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [28]:
# test set(기쁨)
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 기쁨)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n', "%.2f" % (co/2880.))

1112111111111011111111111111211111111111221111111121211111111111111111211111111111111111111111111111111111111111011111111111111111111111111110111111111111111111111111111111111111011111101111111111111211111211111111111112101111111111111111121111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111121111111111111111111111111111112111212111111111111111111111121111111111111111111111111111111111111111101111111112122111111111111111111211111211111111111111111112111111111111111111111111111111101111211111111111111101112111211111101111121112111111111111111111121112111111111111111111111111111111111111111111111101111101111111111111121111111111111111111101111111111111111111111111111111111111111111111111111111111111011111111111111112111111111111111111111111111111111111111111111111111121211111121122111112011111111111111111111121111110111111111111111111111111111121211111111111111111111101111111111111121111111211111111111111111111111111111111111

In [29]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 무표정)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

1112111111111011111111111111211111111111221111111121211111111111111111211111111111111111111111111111111111111111011111111111111111111111111110111111111111111111111111111111111111011111101111111111111211111211111111111112101111111111111111121111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111121111111111111111111111111111112111212111111111111111111111121111111111111111111111111111111111111111101111111112122111111111111111111211111211111111111111111112111111111111111111111111111111101111211111111111111101112111211111101111121112111111111111111111121112111111111111111111111111111111111111111111111101111101111111111111121111111111111111111101111111111111111111111111111111111111111111111111111111111111011111111111111112111111111111111111111111111111111111111111111111111121211111121122111112011111111111111111111121111110111111111111111111111111111121211111111111111111111101111111111111121111111211111111111111111111111111111111111

In [30]:
# test set(찡그림)
read_dir1(test_path3)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 찡그림)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 2:
        co+=1

print('\n', "%.2f" % (co/2880.))

1112111111111011111111111111211111111111221111111121211111111111111111211111111111111111111111111111111111111111011111111111111111111111111110111111111111111111111111111111111111011111101111111111111211111211111111111112101111111111111111121111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111121111111111111111111111111111112111212111111111111111111111121111111111111111111111111111111111111111101111111112122111111111111111111211111211111111111111111112111111111111111111111111111111101111211111111111111101112111211111101111121112111111111111111111121112111111111111111111111111111111111111111111111101111101111111111111121111111111111111111101111111111111111111111111111111111111111111111111111111111111011111111111111112111111111111111111111111111111111111111111111111111121211111121122111112011111111111111111111121111110111111111111111111111111111121211111111111111111111101111111111111121111111211111111111111111111111111111111111

In [0]:
# 5만
# Total params : 3,256,515
# 28s 2ms/step
# acc : 기쁨(0.7194444444444444), 무표정(0.03125), 찡그림(0.24930555555555556)

In [ ]:
# 4만
# Total params: 2,319,427
# 22s 2ms/step
# acc : 기쁨(0.94), 무표정(0.02), 찡그림(0.05)